## Declaraciones

In [2]:
# import
import pandas as pd
import numpy as np

# Joanne

---

# Daniel

---

# Joel

In [4]:
years = ['2009','2010','2011','2012','2013','2014','2015','2016']


In [5]:
#urgencies = pd.DataFrame(columns=['ID','CLUES','MOTATE','TIPOURGENCIA','YEAR'])
urgencies = pd.DataFrame(columns=['CLUES','MOTATE','TIPOURGENCIA','YEAR','tmp_trail'])

for year in years:
    path = f'Datos/Urgencias_{year}/'
    file = f'URGENCIAS_{year}.csv'
    try:
        
        if year in ['2009']:
            print(f'[LOG]: Processing... {path+file}')
            urgencies_tmp = pd.read_csv(path+file, sep=";", header=None, names=['ID','CLUES','FOLIO','FECHAALTA',
                    'EDAD','CVEEDAD','SEXO',
                    'MUNRESIDENCIA','DERHAB','TIPOURGENCIA', 'MOTATE',
                    #'TIPOCAMA','ENVIADOA','MP',
                    'AFECPRIN','IRA','PLANEDA','SOBRESEDA',
                    'FECHAINGRESO','HORASESTANCIA','MES_ESTADISTICO',
                    'HORAINIATE','MININIATE','HORATERATE','MINTERATE'],
                    dtype={'ID': object,'CLUES': object,'FOLIO': object,'FECHAALTA': object,
                    'EDAD': np.float64,'CVEEDAD': object,'SEXO': object,
                    'MUNRESIDENCIA': object,'DERHAB': object,'TIPOURGENCIA': np.float64, 'MOTATE': np.float64,
                    #'TIPOCAMA': np.float64,'ENVIADOA': object,'MP': object,
                    'AFECPRIN': object,'IRA': object,'PLANEDA': object,'SOBRESEDA': object,
                    'FECHAINGRESO': object,'HORASESTANCIA': object,'MES_ESTADISTICO': object,
                    'HORAINIATE': object,'MININIATE': object,'HORATERATE': object,'MINTERATE': object})
            print(f'[LOG]: {file} processed.')
        elif year in ['2010','2011']:
            print(f'[LOG]: Processing... {path+file}')
            urgencies_tmp = pd.read_csv(path+file, sep=";", header=None, names=['ID','CLUES','FOLIO','FECHAALTA',
                    'EDAD','CVEEDAD','SEXO',
                    'MUNRESIDENCIA','DERHAB','TIPOURGENCIA',
                    'MOTATE','TIPOCAMA','ENVIADOA','MP',
                    'AFECPRIN','IRA','PLANEDA','SOBRESEDA',
                    'FECHAINGRESO','HORASESTANCIA','MES_ESTADISTICO',
                    'HORAINIATE','MININIATE','HORATERATE','MINTERATE'],
                    dtype={'ID': object,'CLUES': object,'FOLIO': object,'FECHAALTA': object,
                    'EDAD': np.float64,'CVEEDAD': object,'SEXO': object,
                    'MUNRESIDENCIA': object,'DERHAB': object,'TIPOURGENCIA': np.float64,
                    'MOTATE': np.float64,'TIPOCAMA': np.float64,'ENVIADOA': object,'MP': object,
                    'AFECPRIN': object,'IRA': object,'PLANEDA': object,'SOBRESEDA': object,
                    'FECHAINGRESO': object,'HORASESTANCIA': np.int32,'MES_ESTADISTICO': np.int32,
                    'HORAINIATE': np.int32,'MININIATE': np.int32,'HORATERATE': np.int32,'MINTERATE': np.int32})
            print(f'[LOG]: {file} processed.')
        elif year in ['2012','2013','2014']:
            print(f'[LOG]: Processing... {path+file}')
            urgencies_tmp = pd.read_csv(path+file, sep=";", header=None, names=['ID','CLUES','FOLIO','FECHAALTA',
                    'EDAD','CVEEDAD','SEXO','ENTRESIDENCIA',
                    'MUNRESIDENCIA','DERHAB','TIPOURGENCIA',
                    'MOTATE','TIPOCAMA','ENVIADOA','MP',
                    'AFECPRIN','IRA','PLANEDA','SOBRESEDA',
                    'FECHAINGRESO','HORASESTANCIA','MES_ESTADISTICO',
                    'HORAINIATE','MININIATE','HORATERATE','MINTERATE'],
                    dtype={'ID': object,'CLUES': object,'FOLIO': object,'FECHAALTA': object,
                    'EDAD': object,'CVEEDAD': object,'SEXO': object,'ENTRESIDENCIA': object,
                    'MUNRESIDENCIA': object,'DERHAB': object,'TIPOURGENCIA': np.float64,
                    'MOTATE': np.float64,'TIPOCAMA': object,'ENVIADOA': object,'MP': object,
                    'AFECPRIN': object,'IRA': object,'PLANEDA': object,'SOBRESEDA': object,
                    'FECHAINGRESO': object,'HORASESTANCIA': object,'MES_ESTADISTICO': object,
                    'HORAINIATE': object,'MININIATE': object,'HORATERATE': object,'MINTERATE': object})
            print(f'[LOG]: {file} processed.')
        else:
            print(f'[LOG]: Processing... {path+file}')
            urgencies_tmp = pd.read_csv(path+file,
                    dtype={'ID': object,'CLUES': object,'FOLIO': object,'FECHAALTA': object,
                    'EDAD': object,'CVEEDAD': object,'SEXO': object,'ENTRESIDENCIA': object,
                    'MUNRESIDENCIA': object,'DERHAB': object,'TIPOURGENCIA': np.float64,
                    'MOTATE': np.float64,'TIPOCAMA': object,'ENVIADOA': object,'MP': object,
                    'AFECPRIN': object,'IRA': object,'PLANEDA': object,'SOBRESEDA': object,
                    'FECHAINGRESO': object,'HORASESTANCIA': object,'MES_ESTADISTICO': object,
                    'HORAINIATE': object,'MININIATE': object,'HORATERATE': object,'MINTERATE': object})
            print(f'[LOG]: {file} processed.')
        

        
        #urgencies_tmp = urgencies_tmp[['ID','CLUES','MOTATE','TIPOURGENCIA']]
        urgencies_tmp = urgencies_tmp[['CLUES','MOTATE','TIPOURGENCIA']]
        urgencies_tmp['YEAR'] = year
        urgencies_tmp['tmp_trail'] = 1
        
        print(f'[LOG]: Selected: {file}')
        
        urgencies = pd.concat([urgencies, urgencies_tmp], ignore_index=True)
        
        
        print(f'[LOG]: Appended: {file}')
        
        
    except:
        print(f'[LOG]: Error: {file}')
    

[LOG]: Processing... Datos/Urgencias_2009/URGENCIAS_2009.csv
[LOG]: URGENCIAS_2009.csv processed.
[LOG]: Selected: URGENCIAS_2009.csv
[LOG]: Appended: URGENCIAS_2009.csv
[LOG]: Processing... Datos/Urgencias_2010/URGENCIAS_2010.csv
[LOG]: URGENCIAS_2010.csv processed.
[LOG]: Selected: URGENCIAS_2010.csv
[LOG]: Appended: URGENCIAS_2010.csv
[LOG]: Processing... Datos/Urgencias_2011/URGENCIAS_2011.csv
[LOG]: URGENCIAS_2011.csv processed.
[LOG]: Selected: URGENCIAS_2011.csv
[LOG]: Appended: URGENCIAS_2011.csv
[LOG]: Processing... Datos/Urgencias_2012/URGENCIAS_2012.csv
[LOG]: URGENCIAS_2012.csv processed.
[LOG]: Selected: URGENCIAS_2012.csv
[LOG]: Appended: URGENCIAS_2012.csv
[LOG]: Processing... Datos/Urgencias_2013/URGENCIAS_2013.csv
[LOG]: URGENCIAS_2013.csv processed.
[LOG]: Selected: URGENCIAS_2013.csv
[LOG]: Appended: URGENCIAS_2013.csv
[LOG]: Processing... Datos/Urgencias_2014/URGENCIAS_2014.csv
[LOG]: URGENCIAS_2014.csv processed.
[LOG]: Selected: URGENCIAS_2014.csv
[LOG]: Appended:

In [ ]:
urgencies

In [6]:
urgencies.head()

,CLUES,MOTATE,TIPOURGENCIA,YEAR,tmp_trail
0,CCSSA000100,4.0,3.0,2009,1
1,CCSSA000100,4.0,3.0,2009,1
2,CCSSA000100,4.0,3.0,2009,1
3,CCSSA000100,4.0,3.0,2009,1
4,CCSSA000100,1.0,1.0,2009,1


In [7]:
urgencies.count()

CLUES           75412116
MOTATE          72899234
TIPOURGENCIA    75412116
YEAR            75412116
tmp_trail       75412116
dtype: int64

In [8]:
urgencies_cdmx = urgencies[(urgencies['CLUES'].str.contains("DF"))]

urgencies_cdmx.count()

CLUES           7017663
MOTATE          6934208
TIPOURGENCIA    7017663
YEAR            7017663
tmp_trail       7017663
dtype: int64

In [9]:

#urgencies_agg = urgencies.groupby(['CLUES','MOTATE','TIPOURGENCIA','YEAR']).count()[['CLUES']]

#urgencies_cdmx.groupby(['CLUES','MOTATE','TIPOURGENCIA','YEAR']).CLUES.agg('count').to_frame('Count').reset_index()

urgencies_cdmx = urgencies_cdmx.groupby(['CLUES','MOTATE','TIPOURGENCIA','YEAR'])["tmp_trail"].count().reset_index(name="count")

urgencies_cdmx.head()

,CLUES,MOTATE,TIPOURGENCIA,YEAR,count
0,DFSSA000053,1.0,1.0,2009,3
1,DFSSA000053,1.0,1.0,2010,1579
2,DFSSA000053,1.0,1.0,2011,1492
3,DFSSA000053,1.0,1.0,2012,1111
4,DFSSA000053,1.0,1.0,2013,1255


In [10]:
urgencies_cdmx.count()

CLUES           2463
MOTATE          2463
TIPOURGENCIA    2463
YEAR            2463
count           2463
dtype: int64

In [11]:
output_file = 'data_output/urgencies_by_date.csv'

urgencies_cdmx.to_csv(output_file, encoding='utf-8')

---

---